In [11]:
using LinearAlgebra, CSV, Random, Tables, JuMP, COSMO, SparseArrays

In [2]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
# Matrix sqrt: via diagonalization and sqrt eigenvalues
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
k = 7;
# All close.
maximum(abs.(raw_data * raw_data .- data))
nodes = CSV.File("../NC-K7-Trace-Nodes.csv") |> Tables.matrix;
bounds = CSV.File("../NC-K7-Trace-Bounds.csv") |> Tables.matrix;
# node_id = 50
node_id = 300
sum(nodes[:, node_id] .== 1)
bounds[:, node_id]
selected_data = findall(nodes[:, node_id] .== 1)

3-element Array{Int64,1}:
 58
 60
 62

In [3]:
data_sq = data .* data;

In [4]:
model = Model(SCS.Optimizer)
@variable(model, S[1:n, 1:n] >= 0, PSD)
@objective(model, Max, LinearAlgebra.tr(data_sq * S))
@constraint(model, [i = 1:n], sum(S[i, :]) .== k)
@constraint(model, LinearAlgebra.tr(S) == k)
optimize!(model)

----------------------------------------------------------------------------
	SCS v2.1.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 20604
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 5151, constraints m = 10404
Cones:	primal zero / dual free vars: 102
	linear vars: 5151
	sd vars: 5151, sd blks: 1
Setup time: 1.18e-02s
SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.00e-01  6.21e-01  7.16e-01 -4.07e+02 -6.70e+01  0.00e+00  6.04e-03 
   100| 9.85e-08  1.84e-07  8.13e-09 -8.15e+01 -8.15e+01  7.92e-16  1.08e+00 
-------------------------

In [63]:
model = COSMO.Model();
P = spzeros(N, N)
q = vec(-data_sq)

row_c = repeat(Matrix(-1.0I, n, n), inner=(1, n))
for i = 1:n
    row_c[i, 1 + (n+1)*(i-1)] = k-1
end
row_c = COSMO.Constraint(row_c, zeros(n), COSMO.ZeroSet)

assemble!(
    model,
    P,
    q,
    [
        # PSD
        COSMO.Constraint(sparse(1.0I, N, N), zeros(N), COSMO.PsdCone),
        # [0, 1] constraint
        COSMO.Constraint(sparse(1.0I, N, N), zeros(N), COSMO.Box(zeros(N), ones(N))),
        # Trace == k
        COSMO.Constraint(vec(Matrix(1.0I, n, n))', -Float64(k), COSMO.ZeroSet),
        row_c,
    ])

In [64]:
result = COSMO.optimize!(model)

>>> COSMO - Results
Status: 

Solved
Iterations: 150 (incl. 0 safeguarding iterations)
Optimal Objective: -42.54
Runtime: 877.13ms
Setup Time: 15.08ms

Avg Iter Time: 5.74ms

In [67]:
sqrt(-result.obj_val)

6.52237090658127

In [68]:
data[1:3,1:3]

3×3 Array{Float64,2}:
  1.0        -0.0461484   0.231178
 -0.0461484   1.0        -0.0671091
  0.231178   -0.0671091   1.0

In [77]:
data[[1:3], [1:3]]

ArgumentError: ArgumentError: invalid index: UnitRange{Int64}[1:3] of type Array{UnitRange{Int64},1}

In [78]:
diagind(data)

1:102:10201

In [66]:
fieldnames(COSMO.Result)

(:x, :y, :s, :obj_val, :iter, :safeguarding_iter, :status, :info, :times)

In [59]:
sqrt(42.54)

6.522269543648131